# Connecting to Spark

This is an iPython notebook.  You can execute a cell by clicking on it and pressing shift-enter.

We can execute spark commands in here directly and get immediate results.

When you open up a notebook with `pys`, you automatically have a variable, `sc`, available.  This is a Spark Context.  It's our starting point for all Spark operations.

In [1]:
print sc

We're going to be using Python with DataFrames, which is only available in Spark 1.3 or later.  We're going to be using a recent version of open source spark.  To use it, you'll have to import the `SQLContext`.

In [2]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)
print sql

# Reading a Cassandra Table

In [33]:
users = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="movielens_small", table="users")
print users

DataFrame[id: string, address: string, age: int, city: string, gender: string, name: string, occupation: string, zip: string]


# Displaying results

If we never perform an operation, our dataframe is never read in.  We can force our dataframe into memory and see it by calling `collect()` or `show()` on it.

In [34]:
users.limit(1).collect()

[Row(id=u'bd8dab96-c2f2-4200-830d-4a44abf376ec', address=u'458 Britton Mews', age=19, city=u'New Katina', gender=u'M', name=u'Chanel Lesch', occupation=u'student', zip=u'02146')]

In [35]:
users.limit(10).show()

+--------------------+--------------------+---+---------------+------+--------------------+----------+-----+
|                  id|             address|age|           city|gender|                name|occupation|  zip|
+--------------------+--------------------+---+---------------+------+--------------------+----------+-----+
|bd8dab96-c2f2-420...|    458 Britton Mews| 19|     New Katina|     M|        Chanel Lesch|   student|02146|
|7ccfd064-d902-402...|03155 Bartoletti ...| 46|    Kesslerberg|     M|     Alyvia Walsh MD| librarian|77008|
|9ce825e1-e803-4ee...|    8515 Taryn Inlet| 47|     Jaslynport|     M|       Caesar Ledner|technician|Y1A6B|
|32d6ba75-335d-42d...|69008 Cruickshank...| 30|      Lake Mona|     M|Miss Tristan Schi...|technician|29379|
|3f43f023-37bc-4b4...|7391 Bartoletti M...| 28|    Gerholdland|     M|           Elna Hahn|  engineer|20770|
|4d653d07-1190-4e0...|  59325 Murazik Rest| 43|     Tobinville|     M|Miss Shanika Schn...| librarian|02324|
|35f64a87-8640-43e.

# Basic Filtering

If we're going to do anything with our data, we need to be able to do a simple task: Filtering.

Here's the syntax for filtering:

In [38]:
users.filter(users.age > 20).limit(1).show()

+--------------------+--------------------+---+-----------+------+---------------+----------+-----+
|                  id|             address|age|       city|gender|           name|occupation|  zip|
+--------------------+--------------------+---+-----------+------+---------------+----------+-----+
|7ccfd064-d902-402...|03155 Bartoletti ...| 46|Kesslerberg|     M|Alyvia Walsh MD| librarian|77008|
+--------------------+--------------------+---+-----------+------+---------------+----------+-----+



There's an alternative syntax for filtering:

In [39]:
user[user.age > 20]

DataFrame[id: string, address: string, age: int, city: string, gender: string, name: string, occupation: string, zip: string]

And of course, a third syntax for filters that have a degree of complexity. 

In [40]:
user.filter("name LIKE 'Dani%'").show()

+--------------------+--------------------+---+--------------+------+--------------+----------+-----+
|                  id|             address|age|          city|gender|          name|occupation|  zip|
+--------------------+--------------------+---+--------------+------+--------------+----------+-----+
|c6d0504b-cae5-41e...|43500 Metro Villa...| 53|  South Harden|     F|Dani Traphagen|     other|94043|
|c62f9926-2be5-47b...|561 Maegan Garden...| 52|     Eldontown|     M| Daniel Glover|  educator|93109|
|836c22ea-7187-4b7...|97642 Luigi Row S...| 60|West Dilanfort|     M| Dani Schiller|   retired|95076|
+--------------------+--------------------+---+--------------+------+--------------+----------+-----+



Try filtering for users named "Jon"

# Selecting specific columns

When you only want to see specific fields in a DataFrame, you will use the `select()` method.  For example:

In [41]:
users.select(users.age)

DataFrame[age: int]

Sometimes you'll want to use a different name for a field than is in the original DataFrame.  For that, you'll want to know about `.alias()`.  For instance:

In [44]:
users.select(users.name, users.age.alias("years"))

DataFrame[name: string, years: int]

When you have a pipeline of DataFrame queries, and need to do a filter, you'll need to either temporarily assign the intermediate DataFrames to a variable or you'll need to use the SQL syntax.  For instance:

In [45]:
users.select(user.name, users.age.alias("years")).filter("years > 10").show()

NameError: name 'user' is not defined

In [46]:
tmp = users.select(users.name, users.age.alias("years"))
tmp[tmp.years > 10].show()

+--------------------+-----+
|                name|years|
+--------------------+-----+
|        Chanel Lesch|   19|
|     Alyvia Walsh MD|   46|
|       Caesar Ledner|   47|
|Miss Tristan Schi...|   30|
|           Elna Hahn|   28|
|Miss Shanika Schn...|   43|
|         Allyn Jones|   24|
|Ms. Paulette Kerluke|   18|
|        Meagan Hoppe|   50|
|     Brooklyn Jacobs|   16|
|      Missie Yost MD|   18|
|      Elbridge Swift|   23|
|     Hettie Wiza DDS|   27|
|      Amil Wisozk MD|   48|
|  Penni O'Conner PhD|   26|
|       Basil Labadie|   25|
|      Deliah McGlynn|   28|
|  Mrs. Shay Anderson|   27|
|Miss Chanel Pfeff...|   20|
|       Seneca Schoen|   68|
+--------------------+-----+
only showing top 20 rows



# Select Expressions

Select expressions allow you to perform various SQL-like operations on your data, still in the JVM.

In [47]:
users.selectExpr("age * 10 as old_age").show()

+-------+
|old_age|
+-------+
|    190|
|    460|
|    470|
|    300|
|    280|
|    430|
|    240|
|    180|
|    500|
|    160|
|    180|
|    230|
|    270|
|    480|
|    260|
|    250|
|    280|
|    270|
|    200|
|    680|
+-------+
only showing top 20 rows



# Convenience functions
When working with DataFrames you'll frequently need access to some convenience functions.  For instance, `explode()` is use when you're working with sets and lists.  It creates 1 row per item in the set. 

In [51]:
movies = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="movielens_small", table="movies")

In [60]:
from pyspark.sql.functions import explode

movies.select(explode(movies.genres), movies.name).show()

+-----------+--------------------+
|        col|                name|
+-----------+--------------------+
|Documentary|Fire on the Mount...|
|     Action|Rumble in the Bro...|
|  Adventure|Rumble in the Bro...|
|      Crime|Rumble in the Bro...|
|    Romance|         Diva (1981)|
|    Mystery|         Diva (1981)|
|   Thriller|         Diva (1981)|
|      Drama|         Diva (1981)|
|     Action|         Diva (1981)|
|      Drama|Magic Hour, The (...|
|     Comedy|      Kingpin (1996)|
|     Comedy|    Sleepover (1995)|
|      Drama|    Sleepover (1995)|
|      Crime|L.A. Confidential...|
|  Film-Noir|L.A. Confidential...|
|    Mystery|L.A. Confidential...|
|   Thriller|L.A. Confidential...|
|     Comedy|Raising Arizona (...|
|      Drama|     Infinity (1996)|
|     Comedy|Kicked in the Hea...|
+-----------+--------------------+
only showing top 20 rows



For queries like the above, it's useful to use our alias command:

In [58]:
movies.select(explode(movies.genres).alias("food"), movies.name).show()

+-----------+--------------------+
|       food|                name|
+-----------+--------------------+
|Documentary|Fire on the Mount...|
|     Action|Rumble in the Bro...|
|  Adventure|Rumble in the Bro...|
|      Crime|Rumble in the Bro...|
|    Romance|         Diva (1981)|
|    Mystery|         Diva (1981)|
|   Thriller|         Diva (1981)|
|      Drama|         Diva (1981)|
|     Action|         Diva (1981)|
|      Drama|Magic Hour, The (...|
|     Comedy|      Kingpin (1996)|
|     Comedy|    Sleepover (1995)|
|      Drama|    Sleepover (1995)|
|      Crime|L.A. Confidential...|
|  Film-Noir|L.A. Confidential...|
|    Mystery|L.A. Confidential...|
|   Thriller|L.A. Confidential...|
|     Comedy|Raising Arizona (...|
|      Drama|     Infinity (1996)|
|     Comedy|Kicked in the Hea...|
+-----------+--------------------+
only showing top 20 rows



Tip: When you refer to `movies.genres`, you're looking at a `Column`.  The api for `Column is here: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column

**Advanced Query:** Try selecting the movies who have the genre "Drama".  You'll need to use `explode()`, `alias()` and a filter.

# A nicer reader

Personally I find needing to code `org.apache.spark.sql.cassandra` everywhere a little annoying.  Here's a couple convenience functions that returns a function (slightly tricky) that can be used to reference tables in a keyspace.  Execute the below block.  You can then refer to tables like such:

`user = reader("user")`

In [62]:
def create_reader(sql):
    def reader(table):
        df = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="movielens_small", table=table)
        return df
    return reader

def create_writer(sql, mode="append"):
    def writer(df, table):
        df.write.format("org.apache.spark.sql.cassandra").\
                 options(table=table, keyspace="movielens_small").save(mode="append")
    return writer

writer = create_writer(sql)
reader = create_reader(sql)

# Data Migrations

One thing Spark is useful for is performing data migrations.  We frequently need to take a table and write out a new structure.  Here's an example where we take the movie table and construct a new table that maps genres to movies.  The `writer()` function takes a dataframe and a table. 

Create this table in CQLSH:

```
CREATE TABLE movies_by_genre (
  genre text,
  id uuid,
  name text,
  avg_rating float,
  primary key(genre, id)
);
```

In [70]:
movies_by_genre = movies.select("id", "name", "avg_rating", explode(movies.genres).alias("genre"))

In [71]:
writer(movies_by_genre, "movies_by_genre")

Now it's your turn.  This migration may be a little tricky.  What we want is a leaderboard where we can quickly view the top movies in a given genre.  

```
CREATE TABLE movie_leaderboard (
  genre text,
  avg_rating float,
  id uuid,
  name text,
  primary key (genre, avg_rating, id)
) with clustering order by (avg_rating desc);
```

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode

# SparkSQL

The programatic interface above is pretty convenient, and in my opinion, fun.  There's another interface that's very convenient if you come from a SQL background: SparkSQL.  SparkSQL supports quite a bit of Hive's SQL dialect.

You can register a table to query with SQL like so:

In [74]:
users.registerTempTable("users")

Try registering your movies DataFrame as `movies`.

How's your SQL?  You can execute queries against the temp tables you've registered.  You can perform JOINs, aggregations, sorting, etc.  For instance:

In [76]:
sql.sql("SELECT * from movies where name LIKE 'Rumble in the Bronx%'")

DataFrame[id: string, avg_rating: float, genres: array<string>, name: string, release_date: date, url: string, video_release_date: date]

Try your hand at a few queries.  Find the IDs of 3 movies you love.  For a more advanced challenge, get a list of all the movies made in the year you were born.  (Hint: LIKE)

In [80]:
# lets load our ratings up 
ratings = reader("ratings_by_user")
ratings.registerTempTable("ratings")

# JOINS and Aggregations

Since we've put our movies and our ratings in tables, we can join them.  Pretty convenient.  We can do various JOINs.  By default, like a RDBMS, the inner join is used, but we also can do LEFT, RIGHT, FULL.  We also have unions and subqueries.  We can perform aggregations on our results as well.  We can take the results of any query (a DataFrame) and use it as a table for future queries.  This is incredibly powerful. 

Full docs: https://spark.apache.org/docs/latest/sql-programming-guide.html#compatibility-with-apache-hive

In [97]:
ratings = " ".join(["SELECT movies.name, ratings.rating from users",
                    "JOIN ratings on users.id = ratings.user_id",
                    "JOIN movies on ratings.movie_id = movies.id ",
                    "WHERE users.name = 'Dani Traphagen'",
                    "ORDER BY rating DESC LIMIT 10"])
sql.sql(ratings).show()

+--------------------+------+
|                name|rating|
+--------------------+------+
|Secrets & Lies (1...|     5|
|        Kolya (1996)|     5|
|Sense and Sensibi...|     5|
|      Titanic (1997)|     5|
|         Emma (1996)|     5|
|As Good As It Get...|     5|
|L.A. Confidential...|     5|
|Shall We Dance? (...|     5|
|Good Will Hunting...|     5|
|        Fargo (1996)|     5|
+--------------------+------+

